# 제3 고지 : 고차 미분 계산
## STEP 31 : 고차 미분(이론 편)

이전 단계에서는 현재 DeZero의 구현을 살펴봤다. 요점은 다음과 같다.

- 계산의 '연결'은 `Function`의 `__call__()`에서 만들어진다.
- 구체적인 순전파 / 역전파 계산은 `Function`을 상속한 클래스의 `forward()` / `backward()`에서 이뤄진다.

여기서 주목해야 할 것은 **계산 그래프의 '연결'이 만들어지는 시점으로, 순전파를 계산**할때 만들어진다는 것이다. 즉,**역전파를 계산할 때는 만들어 지지 않는 다**는 것이 문제의 핵심이다.  

### 31.1 역전파 계산 

순전파와 마찬가지로 역전파에도 구체적인 계산 로직이 있다. 다음의 `Sin`함수를 예로 들면,  

```python
class Sin(Function):
    def forward(self,x):
        y= np.sin(x)
        return y 
    
    def backward(self,gy):
        x = self.inputs[0].data 
        gx = gy * np.cos(x)
        return gx 

def sin(x):
    return Sin()(x)
```

`gx = gy * np.cos(x)` 와 같이 구체적인 계산이 이뤄지지만, **현재의 DeZero는 계산과 관련한 연결그래프를 만들지 않는다.**.  그 이유는 해당 계산에서 `Variable` 인스턴스가 아닌 `ndarray`  인스턴스가 사용되기 때문이다.  
만약 역전파 계산시에도 '연결'이 만들어진다면 고차미분을 자동으로 수행할 수 있게 된다. 

이를 이해하기 위해 $y=\sin(x)$ 의 미분을 구하는 계산그래프(`gx = gy * np.cos(x)`)를 살펴보면, 이와 같이 계산 그래프만 존재한다면 `gx.backward()`를 통해 2차 미분 , 나아가 고차미분을 연결하여 역전파를 진행할 수 있다.

![image](../assets/%EA%B7%B8%EB%A6%BC%2031-2.png)


### 31.2 역전파로 계산 그래프 만들기

현재의 DeZero는 **`Variable` 인스턴스를 사용하여 순전파시 '연결'을 만든다.** 이 말은 **역전파시에도 `ndarray` 인스턴스가 아닌 `Variable` 인스턴스를 사용하면 '연결'이 가능**하다는 것이다.  
이를 위해 우선 다음과 같이 **미분값(기울기)를 `Variable` 인스턴스로 유지** 해야한다.

![image](../assets/%EA%B7%B8%EB%A6%BC%2031-3.png)

이렇게 변경하면, 방금 살펴봤던 $y=\sin(x)$ 의 계산 그래프는 다음과 같이 표현 할 수 있다.

<p align='center'>
    <img src='../assets/%EA%B7%B8%EB%A6%BC%2031-4.png' align='center' width='35%'>
    <img src='../assets/%EA%B7%B8%EB%A6%BC%2031-5.png' align='center' width='32%'>
</p>


위의 그림은 `Sin` 클래스의 순전파, 역전파를 수행한 후의 계산 그래프이다. 중요한 것은 **역전파 시에도 계산 그래프가 만들어 진다는 것**이다.  
(여기서 `y`는 함수가 만들어 낸 변수이므로 미분값을 유지하지 않기 때문에 `y.grad`에서 `gy` 로의 참조는 없다.)

